# IMPORTAÇÃO E DOWNLOAD DOS PESOS

In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
import keras.applications.vgg16 as vgg16
from keras.applications.vgg19 import VGG19
import keras.applications.vgg19 as vgg19
from keras.applications.xception import Xception
import keras.applications.xception as xception
from keras.applications.resnet50 import ResNet50
import keras.applications.resnet50 as resnet

import numpy as np
from keras.models import Model

model = VGG16(weights='imagenet', include_top=True)
layer_name = 'fc2'

#model = VGG19(weights='imagenet', include_top=True)
#layer_name = 'fc2'

#model = Xception(weights='imagenet', include_top=True)
#layer_name = 'avg_pool'

#model = ResNet50(weights='imagenet', include_top=True)
#layer_name = 'avg_pool'

intermediate_layer_model = Model(inputs=model.input,
                                     outputs=model.get_layer(layer_name).output)
   
#model.summary()


# CAPTURANDO A  BASE DE DADOS

In [ ]:
import glob
import pandas as pd

path_database = 'C:\\Users\\Renesio\\Documents\\TCC2\\tes\\'
path_write = 'C:\\Users\\Renesio\\Documents\\TCC2\\resultados_TL\\RESULTADO_1000VGG16.csv'

classes = glob.glob(path_database + '*')

# EXTRAINDO CARACTERISTICAS

In [ ]:
def extract_feature_one_image_VGG16(img_path,intermediate_layer_model,
                              input_img=224):
    img = image.load_img(img_path, target_size=(input_img, input_img))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = vgg16.preprocess_input(img_data)
    features = intermediate_layer_model.predict(img_data)
    return features

In [ ]:
import glob
import pandas as pd

classes = glob.glob(path_database + '*')
cont_classe = 0
cont = 0
features_labels_two_classes = pd.DataFrame()
for classe in classes:
    features = []
    images_names = glob.glob(classe + '/*.*')
    print('\r', cont_classe, end='')
    print("\n")
    for image_name in images_names:
        print('\r', cont, end='')
        features.append(extract_feature_one_image_VGG16(image_name,
                                                         intermediate_layer_model))
        
        cont+=1
    features_labels = np.asarray(features)
    features_labels = features_labels.reshape((features_labels.shape[0],
                                               features_labels.shape[2]))
    features_labels = pd.DataFrame(data=features_labels)
    features_labels['Labels'] = np.uint8(cont_classe)
    features_labels_two_classes = features_labels_two_classes.append(features_labels, 
                                                                ignore_index=True)
    cont_classe+=1

features_labels_two_classes.to_csv(path_write,index=False)
print('Terminou...!!')